In [0]:
dbutils.widgets.text("yyyymmdd", "")
yyyymmdd = dbutils.widgets.get("yyyymmdd")

export_all_output = False
debug = True

In [0]:
import re
import os
import shutil
import datetime as dt
import pytz

nz_tz = pytz.timezone("Pacific/Auckland")
def is_valid_yyyymmdd(date_str):
    return bool(re.fullmatch(r"\d{4}(0[1-9]|1[0-2])(0[1-9]|[12]\d|3[01])", date_str))

if not is_valid_yyyymmdd(yyyymmdd): yyyymmdd = (dt.datetime.now(nz_tz) - dt.timedelta(days=1)).strftime("%Y%m%d")

print(f"Daily run for date: {yyyymmdd}")

In [0]:
# src = "/Volumes/ea_dev/sourcefiles/datalake/vSPD"
src = "/Workspace/Users/tuong.nguyen@ea.govt.nz/vSPD"
dst = "/tmp/vSPD"
shutil.copytree(src, dst,dirs_exist_ok=True)
if debug: os.listdir('/tmp/vSPD')

In [0]:
src_folder = f"/Volumes/ea_prd/sourcefiles/datalake/vspd/gdx/vspd_daily_run_settings/{yyyymmdd}/"
dst_folder = "/tmp/vSPD/Programs/"

# Ensure the destination folder exists
os.makedirs(dst_folder, exist_ok=True)

# Loop through all files in the source folder
for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

if debug: os.listdir('/tmp/vSPD/Programs/')

In [0]:
# Define the file path
file_path = '/tmp/vSPD/Programs/mergeGDXsettings.inc'
with open(file_path, "r") as file: content = file.read().replace("\\", "/")
with open(file_path, "w") as file: file.write(content)
if debug: os.system("cat /tmp/vSPD/Programs/mergeGDXsettings.inc")

In [0]:
file_path = '/tmp/vSPD/Programs/vSPDsettings.inc'
with open(file_path, "r") as file: content = file.read().replace("\\", "/")
with open(file_path, "w") as file: file.write(content)
if debug: os.system("cat /tmp/vSPD/Programs/vSPDsettings.inc")

In [0]:
import pandas as pd
csv_file_path = '/tmp/vSPD/Programs/gdxlist.csv'

# Read the CSV into a DataFrame
df = pd.read_csv(csv_file_path,header=None)

# Convert the DataFrame to a list of lists (one list per row)
gdxlist = df.values.tolist()
gdxlist1 = [item for sublist in gdxlist for item in sublist]
if debug:
  print("\n".join(gdxlist1))

In [0]:
src_folder = "/Volumes/ea_prd/sourcefiles/datalake/vspd/gdx/RTD/20250313/"
dst_folder = "/tmp/vSPD/Input/20250313"
os.makedirs(dst_folder, exist_ok=True)

# Loop through all files in the RTD folder and copy all
for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)


src_folder = "/Volumes/ea_prd/sourcefiles/datalake/vspd/gdx/PRSS/20250313/"
dst_folder = "/tmp/vSPD/Input/"
# Loop through all files in the PRSS folder and copy only required file
for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if filename in gdxlist:
        shutil.copy(src_file, dst_file)

src_folder = "/Volumes/ea_prd/sourcefiles/datalake/vspd/gdx/PRSS/20250312/"
dst_folder = "/tmp/vSPD/Input/"
# Loop through all files in the PRSS folder and copy only required file
for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if filename in gdxlist:
        shutil.copy(src_file, dst_file)

if debug: 
  print("\n".join(os.listdir("/tmp/vSPD/Input/20250313/")))

In [0]:
import subprocess

programs_dir = '/tmp/vSPD/Programs'
os.chdir(programs_dir)

# Path to your GAMS executable
gams_path = '/dbfs/GAMS/gams48.6_linux_x64_64_sfx/gams'

# Command to run
command = f"{gams_path} merge_gdx.gms"

# Run the command using subprocess
subprocess.run(command, shell=True, check=True, capture_output=True)

# Command to run
command = f"{gams_path} runvSPD.gms"

# Run the command using subprocess
subprocess.run(command, shell=True, check=True)

In [0]:
dst_folder = f"/Volumes/ea_dev/sourcefiles/datalake/vspd/gdx/vspd_daily_output/{yyyymmdd}/"
os.makedirs(dst_folder, exist_ok=True)

# Copy gdx input file to destination
src_file = f"/tmp/vSPD/Input/Pricing_{yyyymmdd}.gdx"
dst_file = os.path.join(dst_folder,f"Pricing_{yyyymmdd}.gdx")
shutil.copy(src_file, dst_file)

# Copy gdx input file to destination
src_file = f"/tmp/vSPD/Programs/lst/Pricing_{yyyymmdd}.lst"
dst_file = os.path.join(dst_folder,f"Pricing_{yyyymmdd}.lst")
shutil.copy(src_file, dst_file)

# Copy all output files to destination
src_folder = f"/tmp/vSPD/Output/daily_check_{yyyymmdd}"

if export_all_output:
  shutil.copytree(src_folder, dst_folder, dirs_exist_ok=True)
else:
  # Loop through all files in the source folder
  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if os.path.isfile(src_file):
      shutil.copy(src_file, dst_file)

if debug: os.listdir('/tmp/vSPD/Programs/')

In [0]:
dbutils.notebook.exit("Done")

#TEST CODE

In [0]:
if debug: 
  os.system("cat /tmp/vSPD/Programs/vSPDsettings.inc")
  os.system("cat /tmp/vSPD/Programs/vSPDfileList.inc")
  os.system("cat /tmp/vSPD/Programs/mergeGDXsettings.inc")
  os.system("cat /tmp/vSPD/Programs/gdxlist.csv")
  os.system("cat /tmp/vSPD/Programs/pricing_case_files.inc")